In [7]:
import requests
import json
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pickle

In [2]:
import nltk
#nltk.download('all')

In [3]:
def noun_stem_analyzer(string):
    st = nltk.stem.lancaster.LancasterStemmer()
    return [st.stem(word) for word, pos in nltk.pos_tag(
        nltk.word_tokenize(string)) if pos == "NN" or pos == "NNP" or pos == "JJ"]

In [4]:
r = requests.get('https://tomcat.apache.org/tomcat-8.5-doc/changelog.html')
soup = BeautifulSoup(r.text , "html.parser")

notes = [x for x in soup.find_all('div', class_='subsection')]
labels = []
versionstr = ''
for x in [x.find('h4').get('id') for x in notes]:
    v = re.search('Tomcat_([0-9\._]+)', x)
    if v:
        versionstr = v.group(1) #re.findall('Tomcat_([0-9\.]+)_.*/(.*)', x)[0][0]
    labels.append([versionstr, x])
items = {}
for x in enumerate(notes):
    uls = x[1].find_all('ul')
    alllists = []
    for v in uls:
        for l in v.find_all('li'):
            alllists.append(l.text)
    version = labels[x[0]][0]
    items[version] = items.get(version, []) + alllists

In [5]:
wordset = {}
for k,v in items.items():
    wordset[k] = [set(noun_stem_analyzer(x)) for x in v]

In [8]:
with open('word_set.bin', 'wb') as f:
    pickle.dump(wordset, f)